In [1]:
import os
import autogen

config_list = [
    {
        "model": "gpt-4",
        "base_url":"http://0.0.0.0:8000/v1",
        "api_key": "qwen",
    }
]

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}


In [4]:
gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

initializer = autogen.UserProxyAgent(
    name="Init",
    code_execution_config={'use_docker': False},
)

coder = autogen.AssistantAgent(
    name="Retrieve_Action_1",
    llm_config=gpt4_config,
    system_message="""你是程序员。给定一个主题，编写程序。
您编写 python/shell 代码来解决任务。将代码包装在指定脚本类型的代码块中。用户无法修改您的代码。因此，不要建议需要其他人修改的不完整代码。如果代码块不打算由执行者执行，请不要使用代码块。
不要在一个响应中包含多个代码块。不要要求其他人复制和粘贴结果。检查执行器返回的执行结果。
如果结果显示存在错误，请修复错误并再次输出代码。建议使用完整代码，而不是部分代码或代码更改。如果错误无法修复，或者即使代码成功执行后仍未解决任务，请分析问题，重新审视您的假设，收集所需的其他信息，并考虑尝试不同的方法。
""",
)


executor = autogen.UserProxyAgent(
    name="Retrieve_Action_2",
    system_message="执行者。执行程序员员编写的代码并报告结果。",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


scientist = autogen.AssistantAgent(
    name="Research_Action_1",
    llm_config=gpt4_config,
    system_message="""你是科学家。请在看到论文的摘要打印后对论文进行分类，并创建一个包含领域、标题、作者、摘要和链接的 markdown 表格""",
)


def state_transition(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is initializer:
        # init -> retrieve
        return coder
    elif last_speaker is coder:
        # retrieve: action 1 -> action 2
        return executor
    elif last_speaker is executor:
        if messages[-1]["content"] == "exitcode: 1":
            # retrieve --(execution failed)--> retrieve
            return coder
        else:
            # retrieve --(execution success)--> research
            return scientist
    elif last_speaker == "Scientist":
        # research -> end
        return None


groupchat = autogen.GroupChat(
    agents=[initializer, coder, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [ ]:
initializer.initiate_chat(
    manager, message="题目：上周的法学硕士申请论文。要求：5 - 10 篇不同领域的论文。"
)